In [5]:
val movieDf = spark.read.format("csv")
                    .option("header", true)
                    .option("inferSchema", true)
                    .option("delimitter", ",")
                    .load("hdfs://localhost:9000/ml-latest-small/movies.csv")

movieDf.printSchema()
movieDf.show(2)

root
 |-- movieId: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = true)

+-------+----------------+--------------------+
|movieId|           title|              genres|
+-------+----------------+--------------------+
|      1|Toy Story (1995)|Adventure|Animati...|
|      2|  Jumanji (1995)|Adventure|Childre...|
+-------+----------------+--------------------+
only showing top 2 rows



movieDf: org.apache.spark.sql.DataFrame = [movieId: int, title: string ... 1 more field]


In [72]:
// create schema
import org.apache.spark.sql.types.{StringType, StructType, DoubleType, IntegerType, LongType, StructField }

val RatingSchema = StructType (
                                List(
                                    StructField("userId", IntegerType, true),
                                    StructField("movieId", IntegerType, true),
                                      StructField("rating", DoubleType, true),
                                      StructField("timestamp", LongType, true)
                                    )
                    )

import org.apache.spark.sql.types.{StringType, StructType, DoubleType, IntegerType, LongType, StructField}
RatingSchema: org.apache.spark.sql.types.StructType = StructType(StructField(userId,IntegerType,true), StructField(movieId,IntegerType,true), StructField(rating,DoubleType,true), StructField(timestamp,LongType,true))


In [7]:
val ratingDf = spark.read.format("csv")
                    .option("header", true)
                    .option("delimitter", ",")
                    .schema(RatingSchema)
                    .load("hdfs://localhost:9000/ml-latest-small/ratings.csv")

ratingDf.printSchema()
ratingDf.show(2)


root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: long (nullable = true)

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
+------+-------+------+---------+
only showing top 2 rows



ratingDf: org.apache.spark.sql.DataFrame = [userId: int, movieId: int ... 2 more fields]


In [9]:
// Spark SQL, temp view/temp table created with in spark session
// we can expose the dataframe as table/temp table/view
// no table space, row space created by spark, instaed it is same dataframe and rdd behind sql
movieDf.createOrReplaceTempView("movies")

In [10]:
// now we can use sql queries on movies
// when we execute sql, we get dataframe
val df = spark.sql("SELECT * FROM movies")
df.printSchema()
df.show(2)
df.explain()

root
 |-- movieId: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = true)

+-------+----------------+--------------------+
|movieId|           title|              genres|
+-------+----------------+--------------------+
|      1|Toy Story (1995)|Adventure|Animati...|
|      2|  Jumanji (1995)|Adventure|Childre...|
+-------+----------------+--------------------+
only showing top 2 rows

== Physical Plan ==
FileScan csv [movieId#27,title#28,genres#29] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[hdfs://localhost:9000/ml-latest-small/movies.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<movieId:int,title:string,genres:string>




df: org.apache.spark.sql.DataFrame = [movieId: int, title: string ... 1 more field]


In [11]:
ratingDf.createOrReplaceTempView("ratings")

In [12]:
val df = spark.sql("SELECT * FROM ratings")
df.printSchema()
df.show(2)
df.explain()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: long (nullable = true)

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
+------+-------+------+---------+
only showing top 2 rows

== Physical Plan ==
FileScan csv [userId#49,movieId#50,rating#51,timestamp#52L] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[hdfs://localhost:9000/ml-latest-small/ratings.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<userId:int,movieId:int,rating:double,timestamp:bigint>




df: org.apache.spark.sql.DataFrame = [userId: int, movieId: int ... 2 more fields]


In [13]:
// DIY spark sql
// select 5 reocrds from movies where convert  movie title  in upper case
// select distinct rating from ratings and sort the rating
val df2 = spark.sql("SELECT movieId, upper(title), genres from movies LIMIT 5")
df2.show()
// asc
val df3 = spark.sql("SELECT DISTINCT(rating) FROM ratings ORDER BY rating")
df3.show()

// desc
val df4 = spark.sql("SELECT DISTINCT(rating) FROM ratings ORDER BY rating DESC")
df4.show()

+-------+--------------------+--------------------+
|movieId|        upper(title)|              genres|
+-------+--------------------+--------------------+
|      1|    TOY STORY (1995)|Adventure|Animati...|
|      2|      JUMANJI (1995)|Adventure|Childre...|
|      3|GRUMPIER OLD MEN ...|      Comedy|Romance|
|      4|WAITING TO EXHALE...|Comedy|Drama|Romance|
|      5|FATHER OF THE BRI...|              Comedy|
+-------+--------------------+--------------------+

+------+
|rating|
+------+
|   0.5|
|   1.0|
|   1.5|
|   2.0|
|   2.5|
|   3.0|
|   3.5|
|   4.0|
|   4.5|
|   5.0|
+------+

+------+
|rating|
+------+
|   5.0|
|   4.5|
|   4.0|
|   3.5|
|   3.0|
|   2.5|
|   2.0|
|   1.5|
|   1.0|
|   0.5|
+------+



df2: org.apache.spark.sql.DataFrame = [movieId: int, upper(title): string ... 1 more field]
df3: org.apache.spark.sql.DataFrame = [rating: double]
df4: org.apache.spark.sql.DataFrame = [rating: double]


In [18]:
val df5 = spark.sql("SELECT * FROM ratings WHERE rating >=4 AND rating <=5")
df5.show(2)

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
+------+-------+------+---------+
only showing top 2 rows



df5: org.apache.spark.sql.DataFrame = [userId: int, movieId: int ... 2 more fields]


In [57]:
val mostPopularMovies = spark.sql("""
                    SELECT movieId, AVG(rating) as avg_rating, COUNT(userId) as total_ratings 
                    FROM ratings
                    GROUP BY(movieId) 
             
                    HAVING avg_rating >= 3.5 AND total_ratings >=100
                    ORDER BY avg_rating DESC
                    """)
mostPopularMovies.show(5)




+-------+-----------------+-------------+
|movieId|       avg_rating|total_ratings|
+-------+-----------------+-------------+
|    318|4.429022082018927|          317|
|    858|        4.2890625|          192|
|   2959|4.272935779816514|          218|
|   1221| 4.25968992248062|          129|
|  48516|4.252336448598131|          107|
+-------+-----------------+-------------+
only showing top 5 rows



mostPopularMovies: org.apache.spark.sql.DataFrame = [movieId: int, avg_rating: double ... 1 more field]


In [58]:
mostPopularMovies.createOrReplaceTempView("most_popular_movies")

In [60]:
val df = spark.sql("""SELECT most_popular_movies.movieId, title, avg_rating, total_ratings
        FROM most_popular_movies
        INNER JOIN movies ON most_popular_movies.movieId = movies.movieId
        ORDER BY total_ratings DESC
        """)
df.show()

+-------+--------------------+------------------+-------------+
|movieId|               title|        avg_rating|total_ratings|
+-------+--------------------+------------------+-------------+
|    356| Forrest Gump (1994)| 4.164133738601824|          329|
|    318|Shawshank Redempt...| 4.429022082018927|          317|
|    296| Pulp Fiction (1994)| 4.197068403908795|          307|
|    593|Silence of the La...| 4.161290322580645|          279|
|   2571|  Matrix, The (1999)| 4.192446043165468|          278|
|    260|Star Wars: Episod...| 4.231075697211155|          251|
|    480|Jurassic Park (1993)|              3.75|          238|
|    110|   Braveheart (1995)| 4.031645569620253|          237|
|    589|Terminator 2: Jud...| 3.970982142857143|          224|
|    527|Schindler's List ...|             4.225|          220|
|   2959|   Fight Club (1999)| 4.272935779816514|          218|
|      1|    Toy Story (1995)|3.9209302325581397|          215|
|   1196|Star Wars: Episod...|4.21563981

df: org.apache.spark.sql.DataFrame = [movieId: int, title: string ... 2 more fields]


In [74]:
val popularMoviesDf = spark.sql("""SELECT mpm.movieId, title, avg_rating, total_ratings
        FROM most_popular_movies mpm
        INNER JOIN movies m ON mpm.movieId = m.movieId
        ORDER BY total_ratings DESC
        """)
popularMoviesDf.show()

+-------+--------------------+------------------+-------------+
|movieId|               title|        avg_rating|total_ratings|
+-------+--------------------+------------------+-------------+
|    356| Forrest Gump (1994)| 4.164133738601824|          329|
|    318|Shawshank Redempt...| 4.429022082018927|          317|
|    296| Pulp Fiction (1994)| 4.197068403908795|          307|
|    593|Silence of the La...| 4.161290322580645|          279|
|   2571|  Matrix, The (1999)| 4.192446043165468|          278|
|    260|Star Wars: Episod...| 4.231075697211155|          251|
|    480|Jurassic Park (1993)|              3.75|          238|
|    110|   Braveheart (1995)| 4.031645569620253|          237|
|    589|Terminator 2: Jud...| 3.970982142857143|          224|
|    527|Schindler's List ...|             4.225|          220|
|   2959|   Fight Club (1999)| 4.272935779816514|          218|
|      1|    Toy Story (1995)|3.9209302325581397|          215|
|   1196|Star Wars: Episod...|4.21563981

popularMoviesDf: org.apache.spark.sql.DataFrame = [movieId: int, title: string ... 2 more fields]


In [75]:
spark.sql("""CREATE OR REPLACE TEMP VIEW most_popular_temp_table AS 
                    SELECT movieId, avg(rating) as avg_ratings, 
                                    count(userId) as total_ratings
                        from ratings 
                        WHERE rating > 1.0
                        group by movieId 
                        having avg_ratings >= 3.5 and total_ratings >= 100
                        order by avg_ratings desc
                    """)


res60: org.apache.spark.sql.DataFrame = []


In [76]:
spark.sql("SELECT * FROM most_popular_temp_table").show(4)

+-------+-----------------+-------------+
|movieId|      avg_ratings|total_ratings|
+-------+-----------------+-------------+
|    318|4.450793650793651|          315|
|    527|4.335680751173709|          213|
|   2959|4.325581395348837|          215|
|   1221|4.311023622047244|          127|
+-------+-----------------+-------------+
only showing top 4 rows



In [77]:
// Get DataFrame from a table
val df = spark.table("most_popular_temp_table")
df.printSchema()
df.show()


root
 |-- movieId: integer (nullable = true)
 |-- avg_ratings: double (nullable = true)
 |-- total_ratings: long (nullable = false)

+-------+------------------+-------------+
|movieId|       avg_ratings|total_ratings|
+-------+------------------+-------------+
|    318| 4.450793650793651|          315|
|    527| 4.335680751173709|          213|
|   2959| 4.325581395348837|          215|
|   1221| 4.311023622047244|          127|
|   2571| 4.308550185873606|          269|
|    858| 4.306282722513089|          191|
|    296| 4.285953177257525|          299|
|  48516| 4.283018867924528|          106|
|     50|  4.26980198019802|          202|
|    260| 4.259036144578313|          249|
|   1198| 4.258883248730965|          197|
|   1213|              4.25|          126|
|   4973| 4.241525423728813|          118|
|  58559| 4.238255033557047|          149|
|   1196| 4.233333333333333|          210|
|   1197| 4.232394366197183|          142|
|   1193|4.2272727272727275|          132|
|   108

df: org.apache.spark.sql.DataFrame = [movieId: int, avg_ratings: double ... 1 more field]


In [78]:
// DIY
// cache popularMoviesDf
// check and print numver of partitions
// use coalesce to reuce to 1
// write to csv file in hadoop in directory /sql-popular-movies
// write to json file in hadoop in a directory /sql-popular-movies-json
// read sql-popular-movies from hadoop using inferSchema
// read sql-popular-movies-json using inferSchema
// explain popularMoviesDf with extended, go through plans..

In [67]:
val popularMoviesCache = popularMoviesDf.cache()
popularMoviesCache.show(5)

+-------+--------------------+-----------------+-------------+
|movieId|               title|       avg_rating|total_ratings|
+-------+--------------------+-----------------+-------------+
|    356| Forrest Gump (1994)|4.164133738601824|          329|
|    318|Shawshank Redempt...|4.429022082018927|          317|
|    296| Pulp Fiction (1994)|4.197068403908795|          307|
|    593|Silence of the La...|4.161290322580645|          279|
|   2571|  Matrix, The (1999)|4.192446043165468|          278|
+-------+--------------------+-----------------+-------------+
only showing top 5 rows



popularMoviesCache: popularMoviesDf.type = [movieId: int, title: string ... 2 more fields]


In [73]:
 popularMoviesDf.rdd.getNumPartitions


res58: Int = 72
